In [1]:
import torch

In [2]:
f = open('file1.txt', 'r').read().splitlines()[1:]

In [3]:
import numpy as np
import pandas as pd
data = pd.DataFrame([(int(x1), x2, float(x3), float(x4)) for (x1,x2,x3,x4) in (x.split(', ') for x in f)])

In [4]:
from matplotlib import pyplot as plt
#data.plot.scatter(x=0, y=3, s=[.01] * len(data))

In [5]:
classes = data[1].unique()
ctoi = {c: i for i, c in enumerate(classes)}
itoc = {i: c for i, c in enumerate(classes)}
ctoi, itoc

({'other': 0,
  'sniper': 1,
  'scout': 2,
  'engineer': 3,
  'heavy': 4,
  'pyro': 5,
  'spy': 6,
  'soldier': 7},
 {0: 'other',
  1: 'sniper',
  2: 'scout',
  3: 'engineer',
  4: 'heavy',
  5: 'pyro',
  6: 'spy',
  7: 'soldier'})

In [6]:
X = torch.tensor([data[2], data[3]]).float()

#Y = torch.tensor([ctoi[c] for c in data[1]]).float()
Y = torch.nn.functional.one_hot(torch.tensor([ctoi[c] for c in data[1]]), len(classes)).float()



In [7]:
Y[1]

tensor([1., 0., 0., 0., 0., 0., 0., 0.])

In [53]:
def split_data_batches(X, Y, ratio=0.8):
    n = int(X.size(1) * ratio)
    return (X[:, :n], Y[:n, :]), (X[:, n:], Y[n:, :])

(X_train, Y_train), (X_test, Y_test) = split_data_batches(X, Y)

X_train.shape, Y_train.shape, X_test.shape, Y_test.shape

(torch.Size([2, 1337384]),
 torch.Size([1337384, 8]),
 torch.Size([2, 334346]),
 torch.Size([334346, 8]))

In [55]:
block_size = 66 * 5
batch_size = 32

# grab random blocks of data from the trainign set
def get_time_series_batches(data, labels, batch_size, window_size):
    num_samples = data.size(1)
    num_windows = num_samples - window_size + 1
    batch_indices = torch.randint(0, num_windows, (batch_size,))
    xs = []
    ys = []
    for idx in batch_indices:
        x = data[:, idx:idx+window_size].T
        y = labels[idx:idx+window_size, :]

        xs.append(x)
        ys.append(y)
        

    return torch.stack(xs), torch.stack(ys)


In [56]:
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = torch.nn.Linear(block_size * 2, 100)
        self.fc2 = torch.nn.Linear(100, 100)
        self.fc3 = torch.nn.Linear(100, Y.shape[1])

    def forward(self, x):
        x = x.reshape(-1, block_size * 2)
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
net = Net()
loss = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=0.001)

In [57]:
for epoch in range(100):
    x, y = get_time_series_batches(X_train, Y_train, batch_size, block_size)
    optimizer.zero_grad()
    out = net(x)
    y_last = y[:, 0, :]
    l = loss(out, y_last)
    l.backward()
    optimizer.step()

    validation_x, validation_y = get_time_series_batches(X_test, Y_test, batch_size, block_size)
    with torch.no_grad():
        out = net(validation_x)
        y_last = validation_y[:, 0, :]
        lval = loss(out, y_last)

        print(f'Epoch {epoch}, loss {l.item()}, validation loss {lval.item()}')

torch.Size([32, 8]) torch.Size([32, 330, 8])
Epoch 0, loss 3.5641918182373047, validation loss 0.03449944034218788
torch.Size([32, 8]) torch.Size([32, 330, 8])
Epoch 1, loss 8.692159652709961, validation loss 28.554826736450195
torch.Size([32, 8]) torch.Size([32, 330, 8])
Epoch 2, loss 29.949996948242188, validation loss 0.11200404912233353
torch.Size([32, 8]) torch.Size([32, 330, 8])
Epoch 3, loss 11.166413307189941, validation loss 0.01115926168859005
torch.Size([32, 8]) torch.Size([32, 330, 8])
Epoch 4, loss 9.728019714355469, validation loss 0.13150426745414734
torch.Size([32, 8]) torch.Size([32, 330, 8])
Epoch 5, loss 5.143156051635742, validation loss 13.794783592224121
torch.Size([32, 8]) torch.Size([32, 330, 8])
Epoch 6, loss 14.633336067199707, validation loss 12.50705337524414
torch.Size([32, 8]) torch.Size([32, 330, 8])
Epoch 7, loss 11.385908126831055, validation loss 3.4647536277770996
torch.Size([32, 8]) torch.Size([32, 330, 8])
Epoch 8, loss 3.133667469024658, validation

In [ ]:
Y